In [1]:
from rhyme_rus.utils.sounds_statistics import SoundsStatistics
from timeit import default_timer as timer
from pathlib import Path
import json
import dill
from rhyme_rus.rhyme import rhyme
import matplotlib.pyplot as plt  


Parsed wiktionary uploaded successfully
Trained Model uploaded successfully
Trained Model uploaded successfully


# df_n_vowels_after_stress

In [5]:
df_n_vowels_after_stress= SoundsStatistics.make_df_n_vowels_after_stress()
df_n_vowels_after_stress

,vowels,sounds,freq
0,8,17,1
1,7,17,1
2,7,16,4
3,6,16,5
4,8,15,1
5,7,15,4
6,5,15,16
7,6,15,29
8,7,14,2
9,4,14,3


In [12]:
def get_vowels_sounds_pairs(df_n_vowels_after_stress):
    length = len(df_n_vowels_after_stress)
    number_of_words = 1
    vowels_sounds_pairs = []
    for i in range(length):
        i_row = df_n_vowels_after_stress.iloc[i]
        n_vowels = i_row.vowels
        n_sounds = i_row.sounds
        vowels_sounds_pairs.append(list((n_vowels, n_sounds, number_of_words)))
    vowels_sounds_pairs.reverse()
    return vowels_sounds_pairs
vowels_sounds_pairs = get_vowels_sounds_pairs(df_n_vowels_after_stress)
vowels_sounds_pairs[:2]    
         

[[1, 1, 1], [2, 2, 1]]

In [13]:
def get_not_null_tuple(pair):
    tuple_word_sound = SoundsStatistics.get_items_by_n_vowels_after_stress(*pair)
    if (tuple_word_sound != [] 
        and tuple_word_sound[0][0].islower() 
        and "-" not in tuple_word_sound[0][0]):
        word = tuple_word_sound[0][0]
        return word
    return get_not_null_tuple(pair)
pair = (4,5,1)
get_not_null_tuple(pair) 

'аудио'

# make_progression_words

In [14]:
def make_progression_words(vowels_sounds_pairs):
    path = "progression_words.pkl"
    if Path(path).exists:
        with open(path, "rb") as f:
           progression_words = dill.load(f)
    else:
        progression_words = []
        for pair in vowels_sounds_pairs:
            word = get_not_null_tuple(pair)
            progression = list((word, pair[:-1]))
            print(progression)
            progression_words.append(progression)
        # one-syllabel word cannot be rhymed
        progression_words = progression_words[1:]
        
        with open("progression_words.pkl", "wb") as f:
            dill.dump(progression_words, f)
    
    return progression_words
    
progression_words = make_progression_words(vowels_sounds_pairs)
progression_words    

[['оу', [2, 2]],
 ['голоску', [1, 2]],
 ['легионе', [2, 3]],
 ['катяхах', [1, 3]],
 ['русл', [1, 4]],
 ['напрягали', [2, 4]],
 ['девиации', [3, 4]],
 ['реестр', [1, 5]],
 ['обитании', [3, 5]],
 ['атавизме', [2, 5]],
 ['аудио', [4, 5]],
 ['кенотафами', [3, 6]],
 ['аурою', [4, 6]],
 ['монстр', [1, 6]],
 ['феминизмов', [2, 6]],
 ['старитесь', [3, 7]],
 ['ехидством', [2, 7]],
 ['намерении', [4, 7]],
 ['мытарствах', [2, 8]],
 ['задержанных', [3, 8]],
 ['отучиваю', [4, 8]],
 ['склеивании', [5, 8]],
 ['затаскивали', [4, 9]],
 ['варварств', [2, 9]],
 ['застраиваются', [5, 9]],
 ['проткнутый', [3, 9]],
 ['абстрактностей', [3, 10]],
 ['предназначившийся', [4, 10]],
 ['сестринств', [2, 10]],
 ['ёкающую', [5, 10]],
 ['выстраиваниями', [6, 11]],
 ['бодрствуйте', [3, 11]],
 ['выпрямившись', [4, 11]],
 ['выглаживающий', [5, 11]],
 ['откалибровывающая', [6, 12]],
 ['действовавши', [4, 12]],
 ['исчерпываемся', [5, 12]],
 ['сестринством', [3, 12]],
 ['захлёбывающийся', [6, 13]],
 ['выбросившийся', [5, 1

# assess_time_of_rhyme_search

In [15]:
def assess_time_of_rhyme_search(progression_words):
    path = "word_length_time.pkl"
    if Path(path).exists:
        with open(path, "rb") as f:
            word_length_time = dill.load(f)
    else:
        data =[]
        with open(path, 'wb') as f:
            dill.dump(data, f)

        word_length_time = []
        # rhymes_output = []
        for word in progression_words:
            
            start = timer()
            output = rhyme(
                word_without_stress = word[0],
                max_length_pat_of_ipa=6,
                list_score_numbers=range(0, 45, 5),
                max_number_hard_sounds_in_one_pat=1)
            # rhymes_output.append(output) 
            end = timer()
            time_of_rhyming = end - start
            
            with open('word_length_time.pkl', 'rb') as f:
                data = dill.load(f)
            
            new_data = list((time_of_rhyming, word))
            print(new_data)   
            data.append(new_data)
            
            with open('word_length_time.pkl', 'wb') as f:
                dill.dump(data, f)
                
            word_length_time.append(new_data)
        
    return word_length_time

word_length_time = assess_time_of_rhyme_search(progression_words)
print(word_length_time)    

[[0.9381322000044747, ['оу', [2, 2]]], [5.810658099995635, ['голоску', [1, 2]]], [29.084640000000945, ['легионе', [2, 3]]], [21.093350900002406, ['катяхах', [1, 3]]], [35.89741939999658, ['русл', [1, 4]]], [29.73231300000043, ['напрягали', [2, 4]]], [42.32391020000068, ['девиации', [3, 4]]], [79.11763739999878, ['реестр', [1, 5]]], [55.87026209999749, ['обитании', [3, 5]]], [60.32940450000024, ['атавизме', [2, 5]]], [74.88726599999791, ['аудио', [4, 5]]], [273.8360310999997, ['кенотафами', [3, 6]]], [111.04155439999886, ['аурою', [4, 6]]], [212.55438069999946, ['монстр', [1, 6]]], [175.44113590000052, ['феминизмов', [2, 6]]], [288.2955965000001, ['старитесь', [3, 7]]], [357.1196204999942, ['ехидством', [2, 7]]], [159.33904020000045, ['намерении', [4, 7]]], [500.28760180000245, ['мытарствах', [2, 8]]], [250.5485245000018, ['задержанных', [3, 8]]], [185.7245036999957, ['отучиваю', [4, 8]]], [181.6372549000007, ['склеивании', [5, 8]]], [441.0508017000029, ['затаскивали', [4, 9]]], [398.19

# drawing chart of word_length_time 


In [26]:
y = [round(time[0]/60, 1) for i, time in enumerate(word_length_time)]
print(len(y))
print("y", y[0])
my_xticks = [str(time[1]) for i, time in enumerate(word_length_time)]
x = [i for i, time in enumerate(word_length_time)]
print(len(x))
print(len(my_xticks)) 


53
y 0.0
53
53


In [ ]:
plt.plot(x, y)
plt.xticks(x, my_xticks, rotation = 60)
plt.ylabel("time of rhyming in sec.")
plt.xlabel("vowels-consonants length of word")
plt.rcParams["figure.figsize"] = (27,10)
for _x, _y in zip(x,y):
    label = _y
    plt.annotate(label, (_x, _y), textcoords="offset points", xytext=(0,10), ha="center")
plt.savefig("depth_time.png", bbox_inches = "tight")
plt.show()


<Figure size 2700x500 with 0 Axes>

# explore rhyme config vs time

In [45]:
def test(**kwargs):
    print(kwargs)
test(a = 1, b = 2)

{'a': 1, 'b': 2}


In [79]:
nested_list_score_numbers = [list(range(0, i, 5)) for i in range(20, 80, 5)]
print("list_score",nested_list_score_numbers) 
list_max_length_pat_of_ipa = list(range(6,18,1))
print("list_max_length_pat_of_ipa", list_max_length_pat_of_ipa) 
list_max_number_hard_sounds_in_one_pat = [1,2]
print("list_max_number_hard_sounds_in_one_pat", list_max_number_hard_sounds_in_one_pat)
#(7,17)
word_without_stress = "освитедельствованиями"
print("word_without_stress", word_without_stress) 

list_score [[0, 5, 10, 15], [0, 5, 10, 15, 20], [0, 5, 10, 15, 20, 25], [0, 5, 10, 15, 20, 25, 30], [0, 5, 10, 15, 20, 25, 30, 35], [0, 5, 10, 15, 20, 25, 30, 35, 40], [0, 5, 10, 15, 20, 25, 30, 35, 40, 45], [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50], [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55], [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60], [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65], [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]]
list_max_length_pat_of_ipa [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
list_max_number_hard_sounds_in_one_pat [1, 2]
word_without_stress освитедельствованиями


In [69]:
def assess_rhyme_config_vs_time(nested_list_score_numbers, list_max_length_pat_of_ipa)  :
    word_without_stress = "отучиваю"
    
    max_number_hard_sounds_in_one_pat = 1
    
    path_time = "otuchivaju_rhyme_config_vs_time.pkl"
    path_rhyme = "otuchivaju_rhymes.pkl"
    
    if Path(path_time).exists() and Path(path_rhyme).exists():
        with open(path_time, "rb") as f:
            rhyme_config_vs_time = dill.load(f)
        with open(path_rhyme, "rb") as f:
            rhymes_outputs = dill.load(f)
    else:
        data = []
        with open(path_time, "wb") as f:
            dill.dump(data, f)
        with open(path_rhyme, "wb") as f:
            dill.dump(data, f)
                
        rhyme_outputs = []
        rhyme_config_vs_time = []
        for max_length_pat_of_ipa in list_max_length_pat_of_ipa:
            for list_score_numbers in nested_list_score_numbers: 
                start = timer()
                output = rhyme(
                    word_without_stress = word_without_stress,
                    max_length_pat_of_ipa = max_length_pat_of_ipa,
                    list_score_numbers = list_score_numbers,
                    max_number_hard_sounds_in_one_pat = max_number_hard_sounds_in_one_pat)
                end = timer()
                time_of_rhyming = end - start
                print(time_of_rhyming)
                
                rhyming_data = tuple((time_of_rhyming, 
                                    word_without_stress, 
                                    max_length_pat_of_ipa, 
                                    list_score_numbers, 
                                    max_number_hard_sounds_in_one_pat))

                rhyme_outputs.append(output)
                rhyme_config_vs_time.append(rhyming_data)

                with open(path_time, "rb") as f:
                    _rhyme_config_vs_time = dill.load(f)
                with open(path_rhyme, "rb") as f:
                    _rhymes_outputs = dill.load(f)
                
                _rhyme_config_vs_time.append(rhyming_data)           
                _rhymes_outputs.append(output)
                
                with open(path_time, "wb") as f:
                    dill.dump(_rhyme_config_vs_time, f)
                with open(path_rhyme, "wb") as f:
                    dill.dump(_rhymes_outputs, f)
                
    return rhyme_config_vs_time, rhyme_outputs



In [71]:
rhyme_config_vs_time, rhyme_outputs = assess_rhyme_config_vs_time(list_score, list_max_length_pat_of_ipa)

Algorythm reduced combinations for оту'чиваю down to 6 sounds after stressed vowel
Algorythm produces patterns, which include only 1 CPU-consuming parameter
Algorythm produces [0, 5, 10, 15] of range(0, 105, 5) possible combinations
9.569443400003365
Algorythm reduced combinations for оту'чиваю down to 6 sounds after stressed vowel
Algorythm produces patterns, which include only 1 CPU-consuming parameter
Algorythm produces [0, 5, 10, 15, 20] of range(0, 105, 5) possible combinations
24.75023799999326
Algorythm reduced combinations for оту'чиваю down to 6 sounds after stressed vowel
Algorythm produces patterns, which include only 1 CPU-consuming parameter
Algorythm produces [0, 5, 10, 15, 20, 25] of range(0, 105, 5) possible combinations
44.02561229999992
Algorythm reduced combinations for оту'чиваю down to 6 sounds after stressed vowel
Algorythm produces patterns, which include only 1 CPU-consuming parameter
Algorythm produces [0, 5, 10, 15, 20, 25, 30] of range(0, 105, 5) possible com

KeyboardInterrupt: 

In [53]:
path_time = "audio_rhyme_config_vs_time.pkl"
path_rhyme = "audio_rhymes.pkl"

if Path(path_time).exists and Path(path_rhyme).exists:
    print("yes")
else:
    print("no")

yes


In [56]:
Path(path_time).exists()

False